In [ ]:
__author__ = 'Roger'

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA,ARMA
import datetime

%matplotlib inline

## draw arima predict result so that we can analysis easily
## output the data

# test for stationarity with adf
def test_stationarity(timeseries):
    # Determing rolling statistics
#     rolmean = pd.rolling_mean(timeseries, window=window)
#     rolstd = pd.rolling_std(timeseries, window=window)

    # Plot rolling statistics
#     orig = plt.plot(timeseries, color='blue',label='Original')
#     mean = plt.plot(rolmean, color='red', label='Rolling Mean')
#     std = plt.plot(rolstd, color='black', label = 'Rolling Std')
#     plt.legend(loc='best')
#     plt.title('Rolling Mean & Standard Deviation')
#     plt.show(block=False)

    # Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    if dfoutput['Test Statistic'] < dfoutput['Critical Value (1%)']:
        return True
    else:
        return False

def load_data_as_npdict():
    shop = {}
    index = {}
    begin_time = datetime.date(2015, 7, 1)
    with open('../data/shopid_day_num.txt', 'r') as f:
        for line in f.readlines():
            item = line.strip().split(',')
            shopId = int(item[0])
            if shopId not in shop:
                shop[shopId] = np.array(item[1:], 'f')

    for i in range(489):
        if begin_time + datetime.timedelta(i) != datetime.date(2015, 12, 12):
            index[i] = begin_time + datetime.timedelta(i)
    return index, shop

index, shop = load_data_as_npdict()

# test = [5,6,10,18,23,65]
# test = [1,2,3,4,5,6]
test = [18]

# time series
pd_index = pd.date_range('2016-9-1', periods=61)
print(pd_index)

# for i in test:
#     df = pd.Series(shop[i][-61:],index = pd_index)
#     pvalue = test_stationarity(df,7)
#     print(pvalue)
    # print(df.head())
#     test_stationarity(df,8)
#     draw_acf_pacf(df)

for i in test:
    df = pd.Series(shop[i][-61:],index = pd_index,dtype='f')
    model = ARIMA(df,(0,0,3))
    results_AR = model.fit(disp=-1, method='css')
    result = results_AR.predict('2016-11-1', '2016-11-14', dynamic=True)
    print(result.values)
    result_s = pd.Series(result)
    
    df['2016-10-20'] = 1
    df_log = np.log(df)
    print(df)
    plt.plot(df)
    plt.plot(df_log, color='red')
#     test_stationarity(df_log, 3)
    
    plt.plot(df)
    plt.title('RSS: %.4f'% np.sum((results_AR.fittedvalues-df)**2))
    plt.plot(results_AR.fittedvalues, color='red')
    plt.plot(result, color='red')
    plt.show()
    
for i in test:
    df = pd.Series(shop[i][-61:],index = pd_index,dtype='f')
#     model = ARIMA(df,(0,0,3))
#     results_AR = model.fit(disp=-1, method='css')
#     result = results_AR.predict('2016-11-1', '2016-11-8', dynamic=True)
#     result_s = pd.Series(result)
#     df.add(result_s)
    
    print(df)

# diff

In [ ]:

# pd_index = pd.date_range('2016-10-1', periods=31)
# df = pd.Series(shop[23][-31:],index = pd_index,dtype='f')
# df_log = np.log(df)
# test_stationarity(df,5)
# test_stationarity(df_log,5)

# difference
def diff_ts(ts, d):
    global shift_ts_list
    #  dynamic predict the difference
    global last_data_shift_list
    shift_ts_list = []
    last_data_shift_list = []
    tmp_ts = ts
    for i in d:
        last_data_shift_list.append(tmp_ts[-i])
        print(last_data_shift_list)
        shift_ts = tmp_ts.shift(i)
        shift_ts_list.append(shift_ts)
        tmp_ts = tmp_ts - shift_ts
    tmp_ts.dropna(inplace=True)
    return tmp_ts

# revive
def predict_diff_recover(predict_value, d):
    if isinstance(predict_value, float):
        tmp_data = predict_value
        for i in range(len(d)):
            tmp_data = tmp_data + last_data_shift_list[-i-1]
    elif isinstance(predict_value, np.ndarray):
        tmp_data = predict_value[0]
        for i in range(len(d)):
            tmp_data = tmp_data + last_data_shift_list[-i-1]
    else:
        tmp_data = predict_value
        for i in range(len(d)):
            try:
                tmp_data = tmp_data.add(shift_ts_list[-i-1])
            except:
                raise ValueError('What you input is not pd.Series type!')
        tmp_data.dropna(inplace=True)
    return tmp_data


In [ ]:
## draw image to show the relations between them
index, shop = load_data_as_npdict()
for i in [3,4,5,8,12,15,21,22]:
    s = shop[i][-30:]
    p = np.zeros(s.shape)
    for ii in range(s.shape[0]):
        if ii == 0:
            p[ii] = p[ii] + s[ii]
        else:
            p[ii] = p[ii - 1] + s[ii]
    plt.plot(s,color='red')
    plt.show()
    plt.plot(p,color='blue')
    plt.show()

In [ ]:
# show the distribute and tend
index, shop = load_data_as_npdict()
for i in [3,4,5,8,12,15,21,22]:
    s = shop[i]
    p = np.zeros(s.shape)
    for ii in range(7,s.shape[0]):
        if s[ii-7] < s[ii]:
            p[ii] = 1
        else:
            p[ii] = 0

    plt.plot(s[-90:],color='red')
    plt.show()
    plt.scatter(range(30),p[-30:],color='blue')
    plt.axis([0,45,-2,2])
    plt.show()

In [ ]:
## define the difference and recover function
def one_step_diff(series ,d = 0):
    if d == 0:
        return series
    size = series.shape[0]
    print(size)
    result = series[:]
    for i in range(size - 1, d - 1, -1):
        result[i] = result[i] - result[i - d]

    return result


def one_step_recover(series, d=0):
    if d == 0:
        return series
    size = series.shape[0]
    result = series[:]
    for i in range(d, size):
        result[i] = result[i] + result[i - d]

    return result

## simple test
pd_index = pd.date_range('2016-10-1', periods=30)
df = pd.Series(np.append(shop[1][-15:], np.zeros(15)),index = pd_index,dtype='f')
print(df)
result = diff_ts(df,[5])
result2 = one_step_diff(df, 5)
print(result)
print(result2)
print(predict_diff_recover(result,[5]))
print(one_step_recover(result2, 5))

In [ ]:
import warnings
warnings.filterwarnings('ignore')
pd_index = pd.date_range('2016-10-11', periods=35)
print(pd_index)

# output the image to analysis the specific shop with arima

for i in [1,2,3,4,700]:
    train = shop[i][-21:]
    
    
    train_mean = np.mean(train)
    for it in range(train.shape[0]):
        if train[it] > train_mean * 3 or train[it] < train_mean * 0.1:
            train[it] = train_mean
    
    train = np.append(shop[i][-21:],np.zeros(14))
    train[:-14] = np.log(train[:-14])
    df = pd.Series(train,index = pd_index, dtype='f')
#     print(df)
    d = 0
    df_diff = df
    while test_stationarity(df_diff) != True:
        d += 1
        df_diff = one_step_diff(df,d)
    
    dd = df_diff[d:-14]
#     print(dd)
#     df = pd.Series(,index = pd_index,dtype='f')
#     df_log = np.log(df)

    P = [0,1,2,3,4,5,6,7,8,9]
    Q = [0,1,2,3,4,5,6,7,8,9]
    p_b = 0
    q_b = 0
    model_b = None
    bic_b = 999999999
    for p in P:
        for q in Q:
            try:
                model = ARMA(dd, order=(p,q))
                result_arma = model.fit(disp=-1, method='css')
                if result_arma.bic < bic_b:
                    p_b = p
                    q_b = q
                    model_b = model
                    bic_b = result_arma.bic
            except:
                continue
        
    print(d,p_b,q_b,bic_b)
    model = model_b
    result_arma = model.fit(disp=-1, method='css')
    predict_ts = result_arma.predict()
    df_diff[-14:] = result_arma.predict('2016-11-1', '2016-11-14', dynamic=True)
    #print(predict_ts)
    #print(df_diff)
    
    df_diff = one_step_recover(df_diff,d)
    
    recover = np.exp(df_diff)
    predict_ts.plot(color='red')
    recover[:-14].plot(color='blue')
    recover[-14:].plot(color='red')
    plt.show()
    #print(recover)
#     re_result = predict_diff_recover(predict_ts, d=[7])
#     result = np.exp(re_result)


#     df.plot(color='blue')
#     result.plot(color='red')
# #     rr = result_arma.predict('2016-11-1', '2016-11-14', dynamic=True)
# #     rr.plot(color='green')
#     plt.show()
#     f = result_arma.predict('2016-11-1', '2016-11-14', dynamic=True)
#     f = result_arma.predict()
#     print(f)
#     print(result_arma.forecast(10))